# Data Processing

In [9]:
from google.colab import drive
drive.mount('/content/drive')

import scipy.io
import numpy as np
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Charging, Discharging and Impedance Data over Cycles

This data has been taken from a Matlab file exported from my Drive folder and seperated based on cycle types. Then, each data frame has been saved to corresponding csv files in the same folder that they were taken.

In [28]:
file_path1 = "/content/drive/MyDrive/dsa210/battery_dataset/B0005.mat"

mat_data = scipy.io.loadmat(file_path1)
b0005 = mat_data['B0005']
cycle_struct = b0005[0, 0]['cycle']

charge_records_B0005 = []
discharge_records_B0005 = []
impedance_records_B0005 = []

for i in range(cycle_struct.shape[1]):
  cycle = cycle_struct[0, i]
  cycle_type = cycle['type'][0]
  amb_temp = cycle['ambient_temperature'].flatten()[0]
  data = cycle['data']

  if cycle_type == "charge":
    voltage = data['Voltage_measured'][0, 0].flatten()
    current = data['Current_measured'][0, 0].flatten()
    temperature = data['Temperature_measured'][0, 0].flatten()
    current_charge = data['Current_charge'][0, 0].flatten()
    voltage_charge = data['Voltage_charge'][0, 0].flatten()
    time = data['Time'][0, 0].flatten()

    for j in range(len(voltage)):
      charge_records_B0005.append({
        "Cycle index": i+1,
        "Ambient temperature (°C)": amb_temp,
        "Time (secs)": time[j],
        "Battery terminal voltage (Volts)": voltage[j],
        "Battery output current (Amps)": current[j],
        "Battery temperature (°C)": temperature[j],
        "Current measured at charger (Amps)": current_charge[j],
        "Voltage measured at charger (Volts)": voltage_charge[j]
      })

  elif cycle_type == "discharge":
    voltage = data['Voltage_measured'][0, 0].flatten()
    current = data['Current_measured'][0, 0].flatten()
    temperature = data['Temperature_measured'][0, 0].flatten()
    current_load = data['Current_load'][0, 0].flatten()
    voltage_load = data['Voltage_load'][0, 0].flatten()
    time = data['Time'][0, 0].flatten()
    capacity = data['Capacity'][0, 0].flatten()[0]

    for j in range(len(voltage)):
      discharge_records_B0005.append({
        "Cycle index": i+1,
        "Ambient temperature (°C)": amb_temp,
        "Battery terminal voltage (Volts)": voltage[j],
        "Battery output current (Amps)": current[j],
        "Battery temperature (°C)": temperature[j],
        "Current measured at load (Amps)": current_load[j],
        "Voltage measured at load (Volts)": voltage_load[j],
        "Time (secs)": time[j],
        "Battery capacity (Ahr) for discharge till 2.7V": capacity
      })

  elif cycle_type == "impedance":
    sense_current = data['Sense_current'][0, 0].flatten()
    battery_current = data['Battery_current'][0, 0].flatten()
    current_ratio = data['Current_ratio'][0, 0].flatten()
    battery_impedance = data['Battery_impedance'][0, 0].flatten()
    rectified_impedance = data['Rectified_Impedance'][0, 0].flatten()
    Re = data['Re'][0, 0].flatten()[0]
    Rct = data['Rct'][0, 0].flatten()[0]

    for j in range(len(sense_current)):
      impedance_records_B0005.append({
        "Cycle index": i+1,
        "Ambient temperature (°C)": amb_temp,
        "Current in sense branch (Amps)": sense_current[j],
        "Current in battery branch (Amps)": battery_current[j],
        "Ratio of the above currents": current_ratio[j],
        "Battery impedance (Ohms) computed from raw data": battery_impedance[j],
        "Calibrated and smoothed battery impedance (Ohms)": rectified_impedance[j] if j < len(rectified_impedance) else None,
        "Estimated electrolyte resistance (Ohms)": Re,
        "Estimated charge transfer resistance (Ohms)": Rct
      })

df_charge_B0005 = pd.DataFrame(charge_records_B0005)
df_discharge_B0005 = pd.DataFrame(discharge_records_B0005)
df_impedance_B0005 = pd.DataFrame(impedance_records_B0005)

charge_output_file = "/content/drive/MyDrive/dsa210/df_charge_B0005.csv"
discharge_output_file = "/content/drive/MyDrive/dsa210/df_discharge_B0005.csv"
impedance_output_file = "/content/drive/MyDrive/dsa210/df_impedance_B0005.csv"

df_charge_B0005.to_csv(charge_output_file, index=False)
print(f"Data has been saved to {charge_output_file}")

df_discharge_B0005.to_csv(discharge_output_file, index=False)
print(f"Data has been saved to {discharge_output_file}")

df_impedance_B0005.to_csv(impedance_output_file, index=False)
print(f"Data has been saved to {impedance_output_file}")

Data has been saved to /content/drive/MyDrive/dsa210/df_charge_B0005.csv
Data has been saved to /content/drive/MyDrive/dsa210/df_discharge_B0005.csv
Data has been saved to /content/drive/MyDrive/dsa210/df_impedance_B0005.csv


In [23]:
print("Charge data sample:")
print(df_charge_B0005.head())

Charge data sample:
   Cycle index  Ambient temperature (°C)  Time (secs)  \
0            1                        24        0.000   
1            1                        24        2.532   
2            1                        24        5.500   
3            1                        24        8.344   
4            1                        24       11.125   

   Battery terminal voltage (Volts)  Battery output current (Amps)  \
0                          3.873017                      -0.001201   
1                          3.479394                      -4.030268   
2                          4.000588                       1.512731   
3                          4.012395                       1.509063   
4                          4.019708                       1.511318   

   Battery temperature (°C)  Current measured at charger (Amps)  \
0                 24.655358                               0.000   
1                 24.666480                              -4.036   
2              

In [24]:
print("Discharge data sample:")
print(df_discharge_B0005.head())

Discharge data sample:
   Cycle index  Ambient temperature (°C)  Battery terminal voltage (Volts)  \
0            2                        24                          4.191492   
1            2                        24                          4.190749   
2            2                        24                          3.974871   
3            2                        24                          3.951717   
4            2                        24                          3.934352   

   Battery output current (Amps)  Battery temperature (°C)  \
0                      -0.004902                 24.330034   
1                      -0.001478                 24.325993   
2                      -2.012528                 24.389085   
3                      -2.013979                 24.544752   
4                      -2.011144                 24.731385   

   Current measured at load (Amps)  Voltage measured at load (Volts)  \
0                          -0.0006                             

In [25]:
print("Impedance data sample:")
print(df_impedance_B0005.head())

Impedance data sample:
   Cycle index  Ambient temperature (°C)  Current in sense branch (Amps)  \
0           41                        24           -1.000000+  1.000000j   
1           41                        24          820.609497- 36.234550j   
2           41                        24          827.242188- 48.231228j   
3           41                        24          827.193481- 56.195717j   
4           41                        24          824.929504- 53.241478j   

   Current in battery branch (Amps)  Ratio of the above currents  \
0             -1.000000+  1.000000j           1.000000+0.000000j   
1            337.091461- 82.920776j           2.320415+0.463305j   
2            330.631561- 70.013718j           2.424193+0.367465j   
3            330.808624- 61.734425j           2.447002+0.286778j   
4            332.682678- 57.629013j           2.434305+0.261646j   

   Battery impedance (Ohms) computed from raw data  \
0                              -0.438926-0.107298j   
1  

## Remaining Useful Life Data over Cycles

In [15]:
file_path2 = '/content/drive/MyDrive/dsa210/battery_RUL.csv'
df = pd.read_csv(file_path2)

print(df.head())

   Cycle_Index  Discharge Time (s)  Decrement 3.6-3.4V (s)  \
0          1.0             2595.30             1151.488500   
1          2.0             7408.64             1172.512500   
2          3.0             7393.76             1112.992000   
3          4.0             7385.50             1080.320667   
4          6.0            65022.75            29813.487000   

   Max. Voltage Dischar. (V)  Min. Voltage Charg. (V)  Time at 4.15V (s)  \
0                      3.670                    3.211           5460.001   
1                      4.246                    3.220           5508.992   
2                      4.249                    3.224           5508.993   
3                      4.250                    3.225           5502.016   
4                      4.290                    3.398           5480.992   

   Time constant current (s)  Charging time (s)   RUL  
0                    6755.01           10777.82  1112  
1                    6762.02           10500.35  1111  
2 

## Impedance Data over State of Charge of the Battery

In [26]:
file_path3 = '/content/drive/MyDrive/dsa210/impedance.csv'
df = pd.read_csv(file_path3)

print(df.head())

  MEASURE_ID  SOC  BATTERY_ID  FREQUENCY_ID  \
0       02_4  100           2             0   
1       02_4   90           2             0   
2       02_4   80           2             0   
3       02_4   70           2             0   
4       02_4   60           2             0   

                            IMPEDANCE_VALUE  
0  (0.110973570048518-0.00547305228273139j)  
1   (0.10776687163617-0.00597670185700764j)  
2   (0.107165211026746-0.0063320957096541j)  
3  (0.106558961363241-0.00648856565733705j)  
4  (0.106086309875063-0.00559046370050435j)  
